In [1]:
# Step 0. Import libraries and custom modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
# Machine learning -----------------------------------------------------
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
# Preprocessing --------------------------------------------------
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# Metrics --------------------------------------------------------------
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
# Exporting ------------------------------------------------------------
import pickle

In [2]:
# Step 1. Read the dataset
# The data this week comes from Coffee Quality Database courtesy of Buzzfeed Data Scientist 
# James LeDoux. The original data can be found on James' github. The data was re-posted to Kaggle.
# These data were collected from the Coffee Quality Institute's review pages in January 2018.
# There is data for both Arabica and Robusta beans, across many countries and professionally 
# rated on a 0-100 scale. All sorts of scoring/ratings for things like acidity, sweetness, fragrance, 
# balance, etc - may be useful for either separating into visualizations/categories or for 
# modeling/recommenders.
# https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-07-07/readme.md
url = 'https://raw.githubusercontent.com/rfordatascience/' + \
'tidytuesday/master/data/2020/2020-07-07/coffee_ratings.csv'
df_raw = pd.read_csv(url)
df_raw.to_csv('../data/raw/coffee.csv')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   total_cup_points       1339 non-null   float64
 1   species                1339 non-null   object 
 2   owner                  1332 non-null   object 
 3   country_of_origin      1338 non-null   object 
 4   farm_name              980 non-null    object 
 5   lot_number             276 non-null    object 
 6   mill                   1021 non-null   object 
 7   ico_number             1180 non-null   object 
 8   company                1130 non-null   object 
 9   altitude               1113 non-null   object 
 10  region                 1280 non-null   object 
 11  producer               1107 non-null   object 
 12  number_of_bags         1339 non-null   int64  
 13  bag_weight             1339 non-null   object 
 14  in_country_partner     1339 non-null   object 
 15  harv

In [3]:
df_raw.sample(10)

,total_cup_points,species,owner,country_of_origin,farm_name,lot_number,mill,ico_number,company,altitude,...,color,category_two_defects,expiration,certification_body,certification_address,certification_contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
1181,79.33,Arabica,juan luis alvarado romero,Guatemala,conquista / morito,NaN,beneficio ixchel,11/23/0122,"unex guatemala, s.a.",NaN,...,Green,6,"January 12th, 2013",Asociacion Nacional Del Café,b1f20fe3a819fd6b2ee0eb8fdc3da256604f1e53,724f04ad10ed31dbb9d260f0dfd221ba48be8a95,m,NaN,NaN,NaN
73,85.42,Arabica,nucoffee,Brazil,fazenda serra de três barras,NaN,NaN,002/1251/0072,nucoffee,1250m,...,Blue-Green,1,"December 2nd, 2012",NUCOFFEE,567f200bcc17a90070cb952647bf88141ad9c80c,aa2ff513ffb9c844462a1fb07c599bce7f3bb53d,m,1250.0,1250.0,1250.0
30,86.58,Arabica,nora zeas,Nicaragua,NaN,NaN,beneficio atlantic condega,790-0013,NaN,NaN,...,NaN,2,"May 18th, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,NaN,NaN,NaN
338,83.58,Arabica,yunnan coffee exchange,China,menglian mang pan professional cooperative,YNC-16029,menglian mang pan professional cooperative,NaN,yunnan coffee exchange,1100,...,Green,4,"April 6th, 2017",Yunnan Coffee Exchange,683fa6014608fc10ff681b0435b0b2dbe6df988f,f6d87a6c04653c569d4911a66f89d5e30ce83b93,m,1100.0,1100.0,1100.0
558,82.92,Arabica,doi tung development project,Thailand,doi tung development project,NaN,NaN,NaN,NaN,NaN,...,NaN,0,"April 13th, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,NaN,NaN,NaN
18,87.25,Arabica,yunnan coffee exchange,China,echo coffee,YNC-06114,echo coffee mill,NaN,yunnan coffee exchange,1450,...,Green,0,"April 7th, 2017",Yunnan Coffee Exchange,683fa6014608fc10ff681b0435b0b2dbe6df988f,f6d87a6c04653c569d4911a66f89d5e30ce83b93,m,1450.0,1450.0,1450.0
712,82.33,Arabica,ipanema coffees,Brazil,rio verde,NaN,ipanema coffees,002/1660/0107,ipanema coffees,1268,...,Green,0,"October 7th, 2016",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1268.0,1268.0,1268.0
123,84.75,Arabica,the coffee source inc.,El Salvador,several,NaN,la majada cooperative,9-29-1986,NaN,1200 mts,...,NaN,3,"August 22nd, 2012",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1200.0,1200.0,1200.0
1017,80.92,Arabica,min hlaing,Myanmar,sithar coffee farm,NaN,local made,unspecified,"sithar coffee co., ltd.",3500 ft,...,NaN,5,"July 16th, 2016",Coffee Quality Institute,1d4c7f93129f9fb1c8a5f0ce0e36cc1cf4c2f4d7,0f62c9236e3ff5c4921da1e22a350aa99482779d,ft,1066.8,1066.8,1066.8
1052,80.67,Arabica,"ceca, s.a.",Costa Rica,gamboa,NaN,beneficio montañas del diamante,5-025-0048,"ceca,s.a.",1.85,...,Blue-Green,4,"February 27th, 2016",Specialty Coffee Association of Costa Rica,8e0b118f3cf3121ab27c5387deacdb7d4d2a60b1,5eb2b7129d9714c43825e44dc3bca9423de209e9,m,185.0,185.0,185.0


In [16]:
# 1.2 Transform data
# Select metrics
df_interim = df_raw.copy()
# Select columns
df_interim = df_interim[['total_cup_points',
                         'species',
                         'country_of_origin',
                         'variety',
                         'aroma',
                         'aftertaste',
                         'acidity',
                         'body',
                         'balance',
                         'sweetness',
                         'altitude_mean_meters',
                         'moisture']]
df_interim = df_interim.dropna()

top_varieties = ['Caturra','Typica','Bourbon','Catuai','Yellow Bourbon'] # En el other vamos a meter el otro
df_interim['variety'] = df_interim['variety'].apply(lambda x: x if x in top_varieties else 'Other')

top_countries = ['Mexico','Guatemala','Colombia','Brazil','Taiwan','Honduras','Costa Rica','Tanzania, United Republic Of']
df_interim['country_of_origin'] = df_interim['country_of_origin'].apply(lambda x: x if x in top_countries else 'Other')

df_interim['specialty'] = df_interim['total_cup_points'].apply(lambda x: 'Yes' if x > 80 else 'No')

df_interim['variety'] = pd.Categorical(df_interim['variety'])
df_interim['country_of_origin'] = pd.Categorical(df_interim['country_of_origin'])
df_interim['species'] = pd.Categorical(df_interim['species'])
df_interim['specialty'] = pd.Categorical(df_interim['specialty'])

#Como hemos creado la columna specialty, podemos botar la columna total_cup_points
df_interim = df_interim.drop(['total_cup_points'], axis=1)

df_interim.to_csv('../data/interim/coffee_model_interim.csv')

df = df_interim.copy()

In [5]:
df_interim['species'].value_counts()

species
Arabica    994
Robusta      3
Name: count, dtype: int64

In [6]:
df_interim['country_of_origin'].value_counts()

country_of_origin
Mexico                          229
Guatemala                       152
Colombia                        126
Brazil                          103
Taiwan                           68
Honduras                         50
Costa Rica                       44
Tanzania, United Republic Of     36
Uganda                           24
Kenya                            22
China                            16
Indonesia                        15
El Salvador                      15
Nicaragua                        14
Ethiopia                         12
Malawi                           11
United States                     9
Myanmar                           8
Thailand                          8
Vietnam                           7
Haiti                             5
Panama                            4
United States (Puerto Rico)       4
Philippines                       4
Laos                              3
India                             2
Peru                              2
Burundi   

In [7]:
df_interim['variety'].value_counts()

variety
Caturra                  238
Typica                   208
Bourbon                  207
Other                    102
Catuai                    69
Yellow Bourbon            33
Mundo Novo                28
Catimor                   20
SL14                      16
SL28                      14
Pacas                     13
Gesha                     12
SL34                       8
Pacamara                   8
Sumatra                    2
Ethiopian Yirgacheffe      2
Mandheling                 2
Java                       2
Arusha                     2
Ruiru 11                   2
Blue Mountain              2
Sumatra Lintong            1
Peaberry                   1
Ethiopian Heirlooms        1
Moka Peaberry              1
Sulawesi                   1
Marigojipe                 1
Pache Comun                1
Name: count, dtype: int64

In [9]:
df_interim.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
total_cup_points,997.0,NaN,NaN,NaN,82.026078,3.688091,0.0,81.17,82.42,83.58,89.92
species,997,2,Arabica,994,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country_of_origin,997,31,Mexico,229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
variety,997,28,Caturra,238,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aroma,997.0,NaN,NaN,NaN,7.558877,0.384985,0.0,7.42,7.58,7.75,8.75
aftertaste,997.0,NaN,NaN,NaN,7.376399,0.404112,0.0,7.17,7.42,7.58,8.58
acidity,997.0,NaN,NaN,NaN,7.514654,0.384867,0.0,7.33,7.5,7.67,8.58
body,997.0,NaN,NaN,NaN,7.496369,0.359624,0.0,7.33,7.5,7.67,8.42
balance,997.0,NaN,NaN,NaN,7.491755,0.414074,0.0,7.33,7.5,7.67,8.58
sweetness,997.0,NaN,NaN,NaN,9.914915,0.541567,0.0,10.0,10.0,10.0,10.0


In [ ]:
#Es necesario hacer una limpieza y poner algunas reglas.

In [10]:
df_interim['country_of_origin'].value_counts(normalize=True) #country_of_origin es una variable categorica. De tipo String
#Se debe tener cuidado con generar tantas columnas en un dataset tan pequeño. No se debe generar una columna para cada categoria de country_of_origin

#Ley de pareto:
#El 20% de los datos representa el 80% de la poblacion. Para reducir las columnas => 0,2296+0,1524+0,1263+0,10+0,068+0,05+0,044 desde Mexico hasta Tanzania

country_of_origin
Mexico                          0.229689
Guatemala                       0.152457
Colombia                        0.126379
Brazil                          0.103310
Taiwan                          0.068205
Honduras                        0.050150
Costa Rica                      0.044132
Tanzania, United Republic Of    0.036108
Uganda                          0.024072
Kenya                           0.022066
China                           0.016048
Indonesia                       0.015045
El Salvador                     0.015045
Nicaragua                       0.014042
Ethiopia                        0.012036
Malawi                          0.011033
United States                   0.009027
Myanmar                         0.008024
Thailand                        0.008024
Vietnam                         0.007021
Haiti                           0.005015
Panama                          0.004012
United States (Puerto Rico)     0.004012
Philippines                     0.00401

In [12]:
df_interim['variety'].value_counts(normalize=True)

#Mismo proceso para obtener el 80% de los datos: 0,2387+0,2086+0,2076+0,1023+0,0692+0,0330 aprox 85%
#Desde Cuturra a Catuai o Yellow Bourbon

variety
Caturra                  0.238716
Typica                   0.208626
Bourbon                  0.207623
Other                    0.102307
Catuai                   0.069208
Yellow Bourbon           0.033099
Mundo Novo               0.028084
Catimor                  0.020060
SL14                     0.016048
SL28                     0.014042
Pacas                    0.013039
Gesha                    0.012036
SL34                     0.008024
Pacamara                 0.008024
Sumatra                  0.002006
Ethiopian Yirgacheffe    0.002006
Mandheling               0.002006
Java                     0.002006
Arusha                   0.002006
Ruiru 11                 0.002006
Blue Mountain            0.002006
Sumatra Lintong          0.001003
Peaberry                 0.001003
Ethiopian Heirlooms      0.001003
Moka Peaberry            0.001003
Sulawesi                 0.001003
Marigojipe               0.001003
Pache Comun              0.001003
Name: proportion, dtype: float64

In [13]:
top_varieties = ['Caturra','Typica','Bourbon','Catuai','Yellow Bourbon'] # En el other vamos a meter el otro
df_interim['variety'] = df_interim['variety'].apply(lambda x: x if x in top_varieties else 'Other')

top_countries = ['Mexico','Guatemala','Colombia','Brazil','Taiwan','Honduras','Costa Rica','Tanzania, United Republic Of']
df_interim['country_of_origin'] = df_interim['country_of_origin'].apply(lambda x: x if x in top_countries else 'Other')

df_interim['specialty'] = df_interim['total_cup_points'].apply(lambda x: 'Yes' if x > 80 else 'No')

df_interim.sample(10)

,total_cup_points,species,country_of_origin,variety,aroma,aftertaste,acidity,body,balance,sweetness,altitude_mean_meters,moisture,specialty
1134,79.92,Arabica,Mexico,Bourbon,7.50,7.00,7.17,7.25,7.08,10.0,1100.00,0.00,No
276,83.83,Arabica,Costa Rica,Caturra,7.67,7.75,7.67,7.50,7.75,10.0,1850.00,0.00,Yes
340,83.58,Arabica,Mexico,Caturra,7.75,7.58,7.58,7.50,7.67,10.0,1200.00,0.12,Yes
815,81.92,Arabica,Guatemala,Bourbon,7.58,7.25,7.58,7.58,7.50,10.0,1901.00,0.10,Yes
1274,75.58,Arabica,Guatemala,Bourbon,7.92,7.17,7.58,7.33,7.17,8.0,1310.64,0.11,No
583,82.75,Arabica,Other,Other,7.50,7.42,7.58,7.58,7.50,10.0,1500.00,0.12,Yes
20,87.25,Arabica,Other,Other,8.00,8.08,8.50,8.25,8.00,10.0,2019.00,0.00,Yes
1171,79.50,Arabica,Colombia,Caturra,7.83,7.42,7.42,7.67,7.50,10.0,1550.00,0.00,No
686,82.42,Arabica,Brazil,Catuai,7.58,7.33,7.50,7.50,7.42,10.0,1000.00,0.12,Yes
155,84.58,Arabica,Colombia,Caturra,7.42,7.75,7.75,8.08,7.83,10.0,1800.00,0.11,Yes


In [15]:
df_interim['variety'] = pd.Categorical(df_interim['variety'])
df_interim['country_of_origin'] = pd.Categorical(df_interim['country_of_origin'])
df_interim['species'] = pd.Categorical(df_interim['species'])
df_interim['specialty'] = pd.Categorical(df_interim['specialty'])

#Como hemos creado la columna specialty, podemos botar la columna total_cup_points
df_interim = df_interim.drop(['total_cup_points'], axis=1)

df_interim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997 entries, 1 to 1333
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   total_cup_points      997 non-null    float64 
 1   species               997 non-null    category
 2   country_of_origin     997 non-null    category
 3   variety               997 non-null    category
 4   aroma                 997 non-null    float64 
 5   aftertaste            997 non-null    float64 
 6   acidity               997 non-null    float64 
 7   body                  997 non-null    float64 
 8   balance               997 non-null    float64 
 9   sweetness             997 non-null    float64 
 10  altitude_mean_meters  997 non-null    float64 
 11  moisture              997 non-null    float64 
 12  specialty             997 non-null    category
dtypes: category(4), float64(9)
memory usage: 82.6 KB
